In [31]:
import pandas as pd
import requests
import cdx_toolkit
from bs4 import BeautifulSoup
from iso3166 import countries
from warcio import ArchiveIterator
import json

To get the meta of a website, We can use urlmeta api

In [32]:
def get_meta(domain):
    url = 'https://api.urlmeta.org/?url={}'.format(domain)
    headers = {
        'Authorization': 'Basic ' + 'YXdlc29tZXByZWV0MTI5OUBnbWFpbC5jb206YWYyeWVqRjN1Rm5WV0JCQ3JjYUo='
    }

    resp = requests.get(url, headers=headers)
    meta = resp.json()
    return meta


In [36]:
url = 'https://www.lifestyle.com.au/'
r = get_meta(url)    

In [37]:
r

{'meta': {'site': {'twitter': 'https://twitter.com/@lifestyletv',
   'favicon': 'https://www.lifestyle.com.au/includes/images/faviconLS.ico?v=6'},
  'keywords': ['LifeStyle',
   'property',
   'diy',
   'gardening',
   'recipes',
   'health',
   'lifestyle channel'],
  'description': 'LifeStyle.com.au is the ultimate lifestyle destination with advice on home and garden, outdoor living, health, fitness, fashion, beauty, food and fishing. Get articles, videos, competitions, expert advice and more.',
  'image': 'https://www.lifestyle.com.au/includes/images/socialLS_new.jpg',
  'type': 'website',
  'title': 'LifeStyle - Inspire & Entertain'},
 'result': {'status': 'OK'}}

<h5>To get the country name, we can  either use "data.similarweb" or "data.alexa". Both gave good results</h5>

In [45]:
c = requests.get('https://data.similarweb.com/api/v1/data?domain=https://www.lifestyle.com.au/')
country = c.json()

In [46]:
iso_code = country['TopCountryShares'][0]['Country']

In [40]:
coun = countries.get(iso_code)

In [41]:
#Gives us the country of the domain
coun_name = coun.name
coun_name

'Australia'

In [6]:
from selenium import webdriver
from bs4 import BeautifulSoup
import random

KEY = "8ed85ca146683131"

def category(url):
    f = open('user_agents.txt')
    lines = f.readlines()
    lines = [line.strip() for line in lines]

    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument('user-agent={}'.format(lines[random.randint(1, 500)]))
    # chrome_options.add_argument('--proxy-server=%s' % PROXY)
    chrome = webdriver.Chrome(chrome_options=chrome_options)
    chrome.get("https://website-categorization.whoisxmlapi.com/api")

    inp = chrome.find_element_by_css_selector('.search-input-wrapper input')
    inp.send_keys(url)
    chrome.find_element_by_css_selector('i.icon-search').click()
    soup = BeautifulSoup(chrome.page_source, 'lxml')

    cat = soup.select('.category span')

    for c in cat:
        print(c.text)

    chrome.close()

In [9]:
category('https://www.aeroplanejelly.com.au')

c:\Users\preet\OneDrive\Desktop\new_start_stanleys\stanleys-env\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: use options instead of chrome_options
  app.launch_new_instance()


Computer and electronics


In [42]:
# To get category of a website, we can use available apis out of which I used https://website-categorization.whoisxmlapi.com/
# Output of the api is shown below

Products/recipes:

- www.aeroplanejelly.com.au -> Food and Drink
- kitchen.nine.com.au -> Food and Drink
- www.bakersdelight.com.au -> Food and Drink
- www.betterbeanco.com -> Beauty and Fitness
- www.bobsredmill.com -> Business And Industry, Computer and electronics, Health
- www.kokoblack.com -> Food and Drink
- golden-boronia.com -> Food and Drink


News/Articles:

- www.texasmonthly.com/ -> People and Society, Science, Reference
- apnews.com/Australia -> News and Media
- www.theage.com.au/ -> Health, Pets and Animals, Sports

In [43]:
# Another service for porviding categories to webistes https://tools.zvelo.com/
# Output of the api is shown below

Products/recipes:

- www.aeroplanejelly.com.au -> Hobbies & Interests - Other
- kitchen.nine.com.au -> Portal Sites, News
- www.bakersdelight.com.au -> Food and Drink
- www.betterbeanco.com -> Information Security
- www.bobsredmill.com -> Food & Drink - Other
- www.kokoblack.com -> Food and Drink
- golden-boronia.com -> Food and Drink


News/Articles:

- www.texasmonthly.com/ -> Magazines
- apnews.com/Australia -> International News
- www.theage.com.au/ -> International News

for https://www.lifestyle.com.au/, Zvelo returns "Television, International News" while website-categorization returns Recreation and Hobbies

### Now if we get the domain of the required category and country, we'll use commoncrawl to getdata 

In [215]:
#We can estimate the number of pages scraped by commoncrawl crawlers
cdx = cdx_toolkit.CDXFetcher(source='cc')
url = 'www.texasmonthly.com/*'

print(url, 'size estimate', cdx.get_size_estimate(url))

www.texasmonthly.com/* size estimate 237000


In [216]:
#getting first 5 objects
objs = []
for obj in cdx.iter(url, limit=5):
    objs.append(obj)

In [217]:
a = objs[1].data
a

{'urlkey': 'com,texasmonthly)/%e2%80%8cpolitics/evictions-rent-payments-unemployment-coronavirus',
 'timestamp': '20200713213922',
 'redirect': 'https://www.texasmonthly.com/%E2%80%8Cpolitics/evictions-rent-payments-unemployment-coronavirus/',
 'status': '301',
 'url': 'https://www.texasmonthly.com/%E2%80%8Cpolitics/evictions-rent-payments-unemployment-coronavirus',
 'mime': 'text/html',
 'digest': '3I42H3S6NNFQ2MSVX7XZKYAYSCX5QBYJ',
 'offset': '45080460',
 'filename': 'crawl-data/CC-MAIN-2020-29/segments/1593657146845.98/crawldiagnostics/CC-MAIN-20200713194203-20200713224203-00128.warc.gz',
 'length': '898',
 'mime-detected': 'text/html'}

In [218]:
#We can form wat and we files path using warc filename

warc_url = 'https://commoncrawl.s3.amazonaws.com/' + objs[0].data['filename']
wet_url = warc_url.replace('/warc/', '/wet/').replace('warc.gz', 'warc.wet.gz')
wat_url = warc_url.replace('/warc/', '/wat/').replace('warc.gz', 'warc.wat.gz')

In [219]:
warc_url

'https://commoncrawl.s3.amazonaws.com/crawl-data/CC-MAIN-2020-29/segments/1593655891884.11/warc/CC-MAIN-20200707080206-20200707110206-00401.warc.gz'

### WARC file

In [220]:
#Getting warc file
r = requests.get(warc_url, stream=True)
records = ArchiveIterator(r.raw)

In [221]:
#First record is warcinfo
record = next(records)
record.rec_type

'warcinfo'

In [222]:
a = record.content_stream().read()
print(a.decode('utf-8'))

isPartOf: CC-MAIN-2020-29
publisher: Common Crawl
description: Wide crawl of the web for July 2020
operator: Common Crawl Admin (info@commoncrawl.org)
hostname: ip-10-67-67-83.ec2.internal
software: Apache Nutch 1.17 (modified, https://github.com/commoncrawl/nutch/)
robots: checked via crawler-commons 1.2-SNAPSHOT (https://github.com/crawler-commons/crawler-commons)
format: WARC File Format 1.1
conformsTo: http://iipc.github.io/warc-specifications/specifications/warc-format/warc-1.1/



In [223]:
#Next is about the request to the server
record = next(records)
record.rec_type

'request'

In [224]:
record.rec_headers.headers

[('WARC-Type', 'request'),
 ('WARC-Date', '2020-07-07T08:17:49Z'),
 ('WARC-Record-ID', '<urn:uuid:8fbee473-4c86-4ffb-83a4-dea0b7dd83bb>'),
 ('Content-Length', '278'),
 ('Content-Type', 'application/http; msgtype=request'),
 ('WARC-Warcinfo-ID', '<urn:uuid:96f24961-e435-4949-beb7-703989de7be3>'),
 ('WARC-IP-Address', '91.220.43.37'),
 ('WARC-Target-URI',
  'http://%e2%80%8bwww.dzintarukoncertzale.lv/events/view/2557')]

In [225]:
#Https headers
record.http_headers.headers

[('User-Agent', 'CCBot/2.0 (https://commoncrawl.org/faq/)'),
 ('Accept', 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8'),
 ('Accept-Language', 'en-US,en;q=0.5'),
 ('Accept-Encoding', 'br,gzip'),
 ('Host', 'www.dzintarukoncertzale.lv'),
 ('Connection', 'Keep-Alive')]

In [226]:
#There is no data in the request
a = record.content_stream().read()
a

b''

In [227]:
#Next is response to the previous request
record = next(records)
record.rec_type

'response'

In [228]:
record.rec_headers.headers

[('WARC-Type', 'response'),
 ('WARC-Date', '2020-07-07T08:17:49Z'),
 ('WARC-Record-ID', '<urn:uuid:8b012124-de19-4236-8736-19176c463a6b>'),
 ('Content-Length', '17079'),
 ('Content-Type', 'application/http; msgtype=response'),
 ('WARC-Warcinfo-ID', '<urn:uuid:96f24961-e435-4949-beb7-703989de7be3>'),
 ('WARC-Concurrent-To', '<urn:uuid:8fbee473-4c86-4ffb-83a4-dea0b7dd83bb>'),
 ('WARC-IP-Address', '91.220.43.37'),
 ('WARC-Target-URI',
  'http://%e2%80%8bwww.dzintarukoncertzale.lv/events/view/2557'),
 ('WARC-Payload-Digest', 'sha1:AGSOKPBOBO6RR4REQCHS2HMJPOYKPPI2'),
 ('WARC-Block-Digest', 'sha1:32XS5LHB66LJ2GHIICTU77OONW5WGF47'),
 ('WARC-Identified-Payload-Type', 'text/html')]

In [229]:
record.http_headers.headers

[('Connection', 'Keep-Alive'),
 ('Set-Cookie', 'session=eb57bf3bac6a69e612ed4abc9b0bd759; path=/'),
 ('Content-Type', 'text/html; charset=utf-8'),
 ('Cache-Control', 'public, max-age=0'),
 ('Expires', 'Tue, 07 Jul 2020 08:17:48 GMT'),
 ('X-Crawler-Transfer-Encoding', 'chunked'),
 ('X-Crawler-Content-Encoding', 'br'),
 ('Vary', 'Accept-Encoding,User-Agent'),
 ('Date', 'Tue, 07 Jul 2020 08:17:48 GMT'),
 ('Server', 'LiteSpeed'),
 ('X-UA-Compatible', 'IE=edge'),
 ('X-Frame-Options', 'DENY'),
 ('Strict-Transport-Security', 'max-age=16070400; includeSubDomains'),
 ('X-Content-Type-Options', 'nosniff'),
 ('X-XSS-Protection', '1; mode=block'),
 ('Content-Length', '16498')]

In [230]:
#Response contains the full html
a = record.content_stream().read()
a

b'<!DOCTYPE html>\n<head>\n    <meta http-equiv="Content-Type" content="text/html; charset=utf-8"/>\n    <title>Dzintaru koncertz\xc4\x81le</title>\n    <meta name="keywords" content="Dzintaru koncertz\xc4\x81le"/>\n    <meta name="description" content="Dzintaru koncertz\xc4\x81le"/>\n    <meta name="robots" content="index, follow"/>\n\n    <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0">\n\n            <!-- facebook share info -->\n        <meta property="fb:app_id" content="781514475514247"/>\n        <meta property="og:site_name" content="Dzintaru Koncertz\xc4\x81le"/>\n        <meta property="og:title" content=\'J\xc5\xabrmalas festiv\xc4\x81ls. Atkl\xc4\x81\xc5\xa1anas koncerts. Velt\xc4\xabjums Marisam Jansonam (P\xc4\x80RCELTS UZ 29. J\xc5\xaaLIJU)\'/>\n\n                    <meta property="og:url"\n              content="http://www.dzintarukoncertzale.lv/test/events/view/2557"/>\n        \

In [231]:
#Next record is the metadata
record = next(records)
record.rec_type

'metadata'

### Some details in metadata
- How long it took to fetch the size
- Detected characterset
- Languages detected

In [232]:
record.rec_headers.headers

[('WARC-Type', 'metadata'),
 ('WARC-Date', '2020-07-07T08:17:49Z'),
 ('WARC-Record-ID', '<urn:uuid:fa32f8ea-106a-4fa3-b10f-02db1f78dffe>'),
 ('Content-Length', '203'),
 ('Content-Type', 'application/warc-fields'),
 ('WARC-Warcinfo-ID', '<urn:uuid:96f24961-e435-4949-beb7-703989de7be3>'),
 ('WARC-Concurrent-To', '<urn:uuid:8b012124-de19-4236-8736-19176c463a6b>'),
 ('WARC-Target-URI',
  'http://%e2%80%8bwww.dzintarukoncertzale.lv/events/view/2557')]

In [233]:
a = record.content_stream().read()
print(a.decode('utf-8'))

fetchTimeMs: 11570
charset-detected: UTF-8
languages-cld2: {"reliable":true,"text-bytes":1229,"languages":[{"code":"lv","code-iso-639-3":"lav","text-covered":0.99,"score":1234.0,"name":"LATVIAN"}]}




In [234]:
# NEXT RECORD
record = next(records)
record.rec_type, record.rec_headers.get_header('WARC-Target-URI')

('request', 'http://%e2%80%8bwww.dzintarukoncertzale.lv/events/view/2719')

In [235]:
record = next(records)
record.rec_type, record.rec_headers.get_header('WARC-Target-URI')

('response', 'http://%e2%80%8bwww.dzintarukoncertzale.lv/events/view/2719')

In [236]:
record = next(records)
record.rec_type, record.rec_headers.get_header('WARC-Target-URI')

('metadata', 'http://%e2%80%8bwww.dzintarukoncertzale.lv/events/view/2719')

In [237]:
r.close()

## Reading WET files

In [271]:
r = requests.get(wet_url, stream=True)
records = ArchiveIterator(r.raw)

In [272]:
#First information is about the crawl

record = next(records)
record.rec_type

'warcinfo'

In [273]:

a = record.content_stream().read()
print(a.decode('utf-8'))

Software-Info: ia-web-commons.1.1.10-SNAPSHOT-20200622102203
Extracted-Date: Sat, 18 Jul 2020 02:04:29 GMT
robots: checked via crawler-commons 1.2-SNAPSHOT (https://github.com/crawler-commons/crawler-commons)
isPartOf: CC-MAIN-2020-29
operator: Common Crawl Admin (info@commoncrawl.org)
description: Wide crawl of the web for July 2020
publisher: Common Crawl




In [274]:
#WET files doesnt contain just the title and text
record = next(records)
record.rec_type

'conversion'

In [275]:
record.rec_headers.headers

[('WARC-Type', 'conversion'),
 ('WARC-Target-URI',
  'http://%e2%80%8bwww.dzintarukoncertzale.lv/events/view/2557'),
 ('WARC-Date', '2020-07-07T08:17:49Z'),
 ('WARC-Record-ID', '<urn:uuid:9490a23e-c3a1-4084-b1d9-dffea8dd17ba>'),
 ('WARC-Refers-To', '<urn:uuid:8b012124-de19-4236-8736-19176c463a6b>'),
 ('WARC-Block-Digest', 'sha1:7CKU5MKZ4U4BKDNFNZYFNEN4CITSTRLE'),
 ('WARC-Identified-Content-Language', 'lav'),
 ('Content-Type', 'text/plain'),
 ('Content-Length', '1391')]

In [276]:
record.http_headers
a = record.content_stream().read()
print(a.decode('utf-8')[:1000])
# The first line is the title of the page. ALl else is the text in the page

Dzintaru koncertzāle
LV EN RU
360°
TŪRE
Aktualitātes
Kalendārs
Dzintaru koncertzāle
Apmeklētājiem
Kontakti
Koncertzāles video
Pasākumu rīkotājiem
Jūrmalas festivāls. Atklāšanas koncerts. Veltījums Marisam Jansonam (PĀRCELTS UZ 29. JŪLIJU)
Ot. 28. Jūlijs 2020 19:30
Lielā zāle
Klausītājiem, kuri iegādājušies biļetes uz 28. jūlija koncertu, piedāvājam apmainīt iepriekš iegādātās biļetes uz pārcelto koncertu 29. jūlijā. Informējam, ka sakarā ar valstī spēkā esošajiem ierobežojumiem, Dzintaru koncertzāles sēdvietu izkārtojums ir mainīts. Lai veiktu biļešu apmaiņu, lūdzam zvanīt Dzintaru koncertzāles biļešu administratoram pa tālruni 26377077 darba dienās no plkst. 9.00 – 17.00.
Apmaināmo biļešu skaits ir ierobežots – 497 vietas (iepriekšējo 2100 vietā). Biļešu apmaiņu iespējams veikt līdz šī gada 17. jūlijam. Tiem, kuri nepaspēs biļetes apmainīt, nauda tiks atgriezta.
Pasākuma organizators: Dzintaru koncertzāle SIA
Rezervēt viesnīcu
Tweet
SIA "Dzintaru koncertzāle"
Turaidas iela 1, Jūrmala,

In [277]:
record = next(records)
record.rec_type

'conversion'

In [278]:
record.rec_headers.get_header('WARC-Target-URI')

'http://%e2%80%8bwww.dzintarukoncertzale.lv/events/view/2719'

In [279]:
a = record.content_stream().read()
print(a.decode('utf-8')[:1000])


Dzintaru koncertzāle
LV EN RU
360°
TŪRE
Aktualitātes
Kalendārs
Dzintaru koncertzāle
Apmeklētājiem
Kontakti
Koncertzāles video
Pasākumu rīkotājiem
LEONĪDA AGUTINA un ANŽELIKAS VARUMAS koncerts (PĀRCELTS UZ 12.06.2021.)
Se. 12. Jūnijs 2021 19:30
Lielā zāle
Koncerts ir pārcelts uz 2020. gada 10. augustu.
Pasākuma organizators: Art Forte SIA
Pasākuma organizators ir atbildīgs par visu sniegto ziņu patiesumu, kas tiek izvietota SIA "Dzintaru koncertzāle" mājas lapā. SIA "Dzintaru koncertzāle" neuzņemas atbildību šajā sakarā.
Pirkt biļetes
Tweet
SIA "Dzintaru koncertzāle"
Turaidas iela 1, Jūrmala, LV-2015
Reģ. Nr. 40003378932
PVN maksātāja Nr. LV 40003378932
AS SEB banka, kods UNLALV2X
Konts LV42UNLA0010009467685
Tālrunis: 67762092
E-pasts: info@dzintarukoncertzale.lv
Atbalstītāji & partneri
Iepirkumi
Izsoles
Presei
Oficiālā informācija
Pieteikties jaunumiem:
Sekot
Tweet
Izstrādāja



In [245]:
r.close()

## WAT File

In [246]:
#WAT files contains metadata

r = requests.get(wat_url, stream=True)
records = ArchiveIterator(r.raw)

In [247]:
#First record is header
record = next(records)

record.rec_type

'warcinfo'

In [248]:
a = record.content_stream().read()
print(a.decode('utf-8'))

Software-Info: ia-web-commons.1.1.10-SNAPSHOT-20200622102203
Extracted-Date: Sat, 18 Jul 2020 02:04:29 GMT
ip: 10.67.67.82
hostname: ip-10-67-67-82.ec2.internal
format: WARC File Format 1.0
conformsTo: http://bibnum.bnf.fr/WARC/WARC_ISO_28500_version1_latestdraft.pdf




In [249]:
#Metadata about the warc records themselves
record = next(records)
record.rec_type

'metadata'

In [250]:
record.rec_headers.headers


[('WARC-Type', 'metadata'),
 ('WARC-Target-URI', 'CC-MAIN-20200707080206-20200707110206-00401.warc.gz'),
 ('WARC-Date', '2020-07-18T02:04:29Z'),
 ('WARC-Record-ID', '<urn:uuid:fc1c5bbd-c288-4e52-86a1-8b950c2a4cb2>'),
 ('WARC-Refers-To', '<urn:uuid:96f24961-e435-4949-beb7-703989de7be3>'),
 ('Content-Type', 'application/json'),
 ('Content-Length', '1234')]

In [251]:
record.http_headers

In [252]:
a = record.content_stream().read()
data = json.loads(a.decode('utf-8'))
data

{'Container': {'Filename': 'CC-MAIN-20200707080206-20200707110206-00401.warc.gz',
  'Compressed': True,
  'Offset': '0',
  'Gzip-Metadata': {'Deflate-Length': '480',
   'Header-Length': '10',
   'Footer-Length': '8',
   'Inflated-CRC': '1030204349',
   'Inflated-Length': '761'}},
 'Envelope': {'Payload-Metadata': {'Actual-Content-Length': '498',
   'Block-Digest': 'sha1:3TWA7432URFSRTLOISLILCQ7LG6JZZEU',
   'Trailing-Slop-Length': '0',
   'Headers-Corrupt': True,
   'Actual-Content-Type': 'application/warc-fields',
   'WARC-Info-Metadata': {'isPartOf': 'CC-MAIN-2020-29',
    'publisher': 'Common Crawl',
    'description': 'Wide crawl of the web for July 2020',
    'operator': 'Common Crawl Admin (info@commoncrawl.org)',
    'hostname': 'ip-10-67-67-83.ec2.internal',
    'software': 'Apache Nutch 1.17 (modified, https://github.com/commoncrawl/nutch/)',
    'robots': 'checked via crawler-commons 1.2-SNAPSHOT (https://github.com/crawler-commons/crawler-commons)',
    'format': 'WARC File 

In [253]:
#next contains metadata about request
record = next(records)
record.rec_type


'metadata'

In [254]:
record.rec_headers.headers


[('WARC-Type', 'metadata'),
 ('WARC-Target-URI',
  'http://%e2%80%8bwww.dzintarukoncertzale.lv/events/view/2557'),
 ('WARC-Date', '2020-07-18T02:04:29Z'),
 ('WARC-Record-ID', '<urn:uuid:40964883-db0b-40bd-88a7-8fac4c9c0c81>'),
 ('WARC-Refers-To', '<urn:uuid:8fbee473-4c86-4ffb-83a4-dea0b7dd83bb>'),
 ('Content-Type', 'application/json'),
 ('Content-Length', '1409')]

In [255]:
# Container shows where the WARC data is, this is about the request


a = record.content_stream().read()
data = json.loads(a.decode('utf-8'))
data

{'Container': {'Filename': 'CC-MAIN-20200707080206-20200707110206-00401.warc.gz',
  'Compressed': True,
  'Offset': '480',
  'Gzip-Metadata': {'Deflate-Length': '447',
   'Header-Length': '10',
   'Footer-Length': '8',
   'Inflated-CRC': '2002798938',
   'Inflated-Length': '658'}},
 'Envelope': {'Payload-Metadata': {'Actual-Content-Type': 'application/http; msgtype=request',
   'HTTP-Request-Metadata': {'Request-Message': {'Method': 'GET',
     'Path': '/events/view/2557',
     'Version': 'HTTP/1.1'},
    'Headers-Length': '276',
    'Headers': {'User-Agent': 'CCBot/2.0 (https://commoncrawl.org/faq/)',
     'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
     'Accept-Language': 'en-US,en;q=0.5',
     'Accept-Encoding': 'br,gzip',
     'Host': 'www.dzintarukoncertzale.lv',
     'Connection': 'Keep-Alive'},
    'Entity-Length': '0',
    'Entity-Digest': 'sha1:3I42H3S6NNFQ2MSVX7XZKYAYSCX5QBYJ',
    'Entity-Trailing-Slop-Length': '0'},
   'Actual-Content-Length

In [256]:
#Http request metadata
data['Envelope']

{'Payload-Metadata': {'Actual-Content-Type': 'application/http; msgtype=request',
  'HTTP-Request-Metadata': {'Request-Message': {'Method': 'GET',
    'Path': '/events/view/2557',
    'Version': 'HTTP/1.1'},
   'Headers-Length': '276',
   'Headers': {'User-Agent': 'CCBot/2.0 (https://commoncrawl.org/faq/)',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'br,gzip',
    'Host': 'www.dzintarukoncertzale.lv',
    'Connection': 'Keep-Alive'},
   'Entity-Length': '0',
   'Entity-Digest': 'sha1:3I42H3S6NNFQ2MSVX7XZKYAYSCX5QBYJ',
   'Entity-Trailing-Slop-Length': '0'},
  'Actual-Content-Length': '278',
  'Block-Digest': 'sha1:TNG7TZ3JLRN45XGOSW6MZIQRCTITMYSB',
  'Trailing-Slop-Length': '4'},
 'Format': 'WARC',
 'WARC-Header-Length': '376',
 'WARC-Header-Metadata': {'WARC-Type': 'request',
  'WARC-Date': '2020-07-07T08:17:49Z',
  'WARC-Record-ID': '<urn:uuid:8fbee473-4c86-4ffb-83a4-dea0b7dd83bb>',


In [257]:
#Next record is about response
record = next(records)
record.rec_type

'metadata'

In [258]:
record.rec_headers.headers


[('WARC-Type', 'metadata'),
 ('WARC-Target-URI',
  'http://%e2%80%8bwww.dzintarukoncertzale.lv/events/view/2557'),
 ('WARC-Date', '2020-07-18T02:04:29Z'),
 ('WARC-Record-ID', '<urn:uuid:369fed4f-4e14-478c-8c04-08f522a80d56>'),
 ('WARC-Refers-To', '<urn:uuid:8b012124-de19-4236-8736-19176c463a6b>'),
 ('Content-Type', 'application/json'),
 ('Content-Length', '6690')]

In [259]:
a = record.content_stream().read()
data = json.loads(a.decode('utf-8'))
data

{'Container': {'Filename': 'CC-MAIN-20200707080206-20200707110206-00401.warc.gz',
  'Compressed': True,
  'Offset': '927',
  'Gzip-Metadata': {'Deflate-Length': '5291',
   'Header-Length': '10',
   'Footer-Length': '8',
   'Inflated-CRC': '1662782872',
   'Inflated-Length': '17691'}},
 'Envelope': {'Payload-Metadata': {'Actual-Content-Type': 'application/http; msgtype=response',
   'HTTP-Response-Metadata': {'Response-Message': {'Status': '200',
     'Version': 'HTTP/1.1',
     'Reason': 'OK'},
    'Headers-Length': '581',
    'Headers': {'Connection': 'Keep-Alive',
     'Set-Cookie': 'session=eb57bf3bac6a69e612ed4abc9b0bd759; path=/',
     'Content-Type': 'text/html; charset=utf-8',
     'Cache-Control': 'public, max-age=0',
     'Expires': 'Tue, 07 Jul 2020 08:17:48 GMT',
     'X-Crawler-Transfer-Encoding': 'chunked',
     'X-Crawler-Content-Encoding': 'br',
     'Vary': 'Accept-Encoding,User-Agent',
     'Date': 'Tue, 07 Jul 2020 08:17:48 GMT',
     'Server': 'LiteSpeed',
     'X-UA

In [260]:
data['Envelope']['Payload-Metadata']

{'Actual-Content-Type': 'application/http; msgtype=response',
 'HTTP-Response-Metadata': {'Response-Message': {'Status': '200',
   'Version': 'HTTP/1.1',
   'Reason': 'OK'},
  'Headers-Length': '581',
  'Headers': {'Connection': 'Keep-Alive',
   'Set-Cookie': 'session=eb57bf3bac6a69e612ed4abc9b0bd759; path=/',
   'Content-Type': 'text/html; charset=utf-8',
   'Cache-Control': 'public, max-age=0',
   'Expires': 'Tue, 07 Jul 2020 08:17:48 GMT',
   'X-Crawler-Transfer-Encoding': 'chunked',
   'X-Crawler-Content-Encoding': 'br',
   'Vary': 'Accept-Encoding,User-Agent',
   'Date': 'Tue, 07 Jul 2020 08:17:48 GMT',
   'Server': 'LiteSpeed',
   'X-UA-Compatible': 'IE=edge',
   'X-Frame-Options': 'DENY',
   'Strict-Transport-Security': 'max-age=16070400; includeSubDomains',
   'X-Content-Type-Options': 'nosniff',
   'X-XSS-Protection': '1; mode=block',
   'Content-Length': '16498'},
  'HTML-Metadata': {'Head': {'Metas': [{'content': 'text/html; charset=utf-8',
      'http-equiv': 'Content-Type'

In [261]:
#Next record contains Metadata of the request
record = next(records)
record.rec_type

'metadata'

In [262]:
record.rec_headers.headers


[('WARC-Type', 'metadata'),
 ('WARC-Target-URI',
  'http://%e2%80%8bwww.dzintarukoncertzale.lv/events/view/2557'),
 ('WARC-Date', '2020-07-18T02:04:29Z'),
 ('WARC-Record-ID', '<urn:uuid:72ca665e-7ea2-4341-9d19-494ba5832fdf>'),
 ('WARC-Refers-To', '<urn:uuid:fa32f8ea-106a-4fa3-b10f-02db1f78dffe>'),
 ('Content-Type', 'application/json'),
 ('Content-Length', '1251')]

In [263]:
# Envelope contains WARC-Metadata, this covers all the actual metadata in the metadata record.


a = record.content_stream().read()
data = json.loads(a)
data

{'Container': {'Filename': 'CC-MAIN-20200707080206-20200707110206-00401.warc.gz',
  'Compressed': True,
  'Offset': '6218',
  'Gzip-Metadata': {'Deflate-Length': '445',
   'Header-Length': '10',
   'Footer-Length': '8',
   'Inflated-CRC': '-955918730',
   'Inflated-Length': '612'}},
 'Envelope': {'Payload-Metadata': {'Actual-Content-Type': 'application/metadata-fields',
   'WARC-Metadata-Metadata': {'Metadata-Records': [{'Name': 'fetchTimeMs',
      'Value': '11570'},
     {'Name': 'charset-detected', 'Value': 'UTF-8'},
     {'Name': 'languages-cld2',
      'Value': '{"reliable":true,"text-bytes":1229,"languages":[{"code":"lv","code-iso-639-3":"lav","text-covered":0.99,"score":1234.0,"name":"LATVIAN"}]}'}]},
   'Actual-Content-Length': '203',
   'Block-Digest': 'sha1:ILU52LZXAC55K5JBOSSW662RKOZ6BFVQ',
   'Trailing-Slop-Length': '0'},
  'Format': 'WARC',
  'WARC-Header-Length': '405',
  'WARC-Header-Metadata': {'WARC-Type': 'metadata',
   'WARC-Date': '2020-07-07T08:17:49Z',
   'WARC-Re